In [1]:
!pip install transformers datasets pandas tokenizers torch scikit-learn

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
import torch

In [4]:
# Load the BBC News dataset
df = pd.read_csv('cnn_dailymail.csv')

# Convert the pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

In [5]:
# Load the T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-base')

def preprocess_data(examples):
    # Add task prefix "summarize: " for T5
    inputs = ["summarize: " + text for text in examples['source_text']]

    # Tokenize input and target text
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')
    labels = tokenizer(examples['target_text'], max_length=128, truncation=True, padding='max_length')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize the dataset
tokenized_dataset = dataset.map(preprocess_data, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have

Map:   0%|          | 0/254869 [00:00<?, ? examples/s]

In [21]:
# Sample 5% of the data
df_sampled, df_unsampled = train_test_split(df, test_size=0.95, random_state=42)

In [22]:
print(df_sampled.columns)

Index(['Unnamed: 0', 'source_text', 'target_text'], dtype='object')


In [23]:
# Split the dataset into training, validation, and test sets
train_data, test_data = train_test_split(df_sampled, test_size=0.1, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42)

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)
test_dataset = Dataset.from_pandas(test_data)

# Tokenize the datasets
train_dataset = train_dataset.map(preprocess_data, batched=True)
val_dataset = val_dataset.map(preprocess_data, batched=True)
test_dataset = test_dataset.map(preprocess_data, batched=True)


Map:   0%|          | 0/10321 [00:00<?, ? examples/s]

Map:   0%|          | 0/1147 [00:00<?, ? examples/s]

Map:   0%|          | 0/1275 [00:00<?, ? examples/s]

In [24]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results_t5_cnn',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
    save_steps=500,
    logging_dir='./logs_cnn',
    logging_steps=100,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [25]:
# Load the T5 model
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

In [27]:
# Fine-tune the model
trainer.train()

# Save the fine-tuned model
trainer.save_model('fine_tuned_t5_cnn_dailymail')
tokenizer.save_pretrained('fine_tuned_t5_cnn_dailymail')

Epoch,Training Loss,Validation Loss
1,1.042300,0.937122
2,0.978300,0.919725
3,0.964100,0.915943


('fine_tuned_t5_cnn_dailymail/tokenizer_config.json',
 'fine_tuned_t5_cnn_dailymail/special_tokens_map.json',
 'fine_tuned_t5_cnn_dailymail/spiece.model',
 'fine_tuned_t5_cnn_dailymail/added_tokens.json')